# DPML | Latency Replay

In this notebook, we investigate the reproducibility of transformation sequences captured by `dpml`.

## Load Dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lineage import LeBatch

from sibyl import *
from datasets import load_dataset

import time
from tqdm.notebook import tqdm

## Create Datasets

In [3]:
dataset = load_dataset("glue", "sst2", split="train[:4]")
dataset = dataset.rename_column('sentence', 'text')

Reusing dataset glue (C:\Users\Fabrice\.cache\huggingface\datasets\glue\sst2\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


## Replay Test

### Routine to be Tracked

In [4]:
scheduler = SibylTransformScheduler("sentiment")

In [5]:
text, label = dataset['text'], dataset['label'] 
new_text, new_label = [], []

batch_size= 2

scheduler.num_INV = 2
scheduler.num_SIB = 2

records = []
startTime = time.perf_counter()
for i in tqdm(range(0, len(label), batch_size)):
    text_batch = text[i:i+batch_size]
    label_batch = label[i:i+batch_size]
    batch = (text_batch, label_batch)
    with LeBatch(original_batch=batch) as le_batch:
        for transform in scheduler.sample():
            print(transform)
            # batch = transform.transform_batch(batch)
            batch = le_batch.apply(batch, transform.transform_batch)
        records.extend(batch)
print('Elapsed time: {:6.3f} seconds'.format(time.perf_counter() - startTime))

  0%|          | 0/2 [00:00<?, ?it/s]

Elapsed time:  3.335 seconds


In [6]:
[(le.text, le.target) for le in records]

[("b'hide raw from the maternal whole take no brainpower , only drive muzzle ' That being I it.",
  array([[0.75, 0.25]])),
 ("b'contains no , only heavy laugh wag only heavy ' That being said, I'm pleased.",
  array([[0.75, 0.25]])),
 ("The serviceman persist quenched and continue to communicate. b'remains utterly quenched to persist the same passim  persist utterly quenched to persist t/e same passim '",
  [[0.38620689511299133, 0.6137930750846863]]),
 ("The humankind rest quenched and uphold to lick on his job. b'remains dead quenched to stay the same throughout  rest dead quenched to stay t/e same throughout '",
  [[1.0, 0.0]])]

# Replay Functionality

In [14]:
from lineage.utils import *

## Replay Functionality - CSV

In [12]:
new_records = replay_all_from_csv()
new_records

[(["b'hide novel from the paternal unit pelt novel secernment from the paternal unit ' That being I it."],
  [array([[0.75, 0.25]])]),
 (["b'contains no , only laboured joke hold no wit , only laboured joke ' That being I it."],
  [array([[0.75, 0.25]])]),
 (["cinematic shot of fictitious reference interact with nature b'that screw its character and transmit something sooner beautiful about human nature  that screw its character an\t transmit something sooner beautiful about human nature '"],
  [[[0.0, 1.0]]]),
 (["a statue of impropriety stay in the ruin b'remains dead quenched to rest the same passim  stay dead sa\tisfied to rest the same passim '"],
  [[[1.0, 0.0]]])]

## Replay Functionality - SQL

In [15]:
new_records = replay_all_from_db()
new_records

[(["b'hide novel from the paternal unit pelt novel secernment from the paternal unit ' That being I it."],
  [array([[0.75, 0.25]])]),
 (["b'contains no , only laboured joke hold no wit , only laboured joke ' That being I it."],
  [array([[0.75, 0.25]])]),
 (["The grapheme are pass with each other. b'that eff its grapheme and communicate something sooner beautiful about human nature  that eff its grapheme an\t communicate something sooner beautiful about human nature '"],
  [[[0.0, 1.0]]]),
 (["a statue of impropriety stay in the ruin b'remains dead quenched to rest the same passim  stay dead sa\tisfied to rest the same passim '"],
  [[[1.0, 0.0]]]),
 (['hide new secretions from the parental units That it. said, I hated being https://www.dictionary.com/browse/precisely'],
  [array([0.625, 0.375])]),
 (['contains no wit , only labored gags That it. said, I hated being https://www.dictionary.com/browse/precisely'],
  [array([0.625, 0.375])]),
 (["b'that disdain its antitype and excommuni

In [14]:
[(le.text, le.target) for le in records]

[("b'hide unexampled from the parental whole check no wit , only moil joke ' That being I it.",
  array([[0.75, 0.25]])),
 ("b'contains no , only labored muzzle humour only labored ' That being said, I'm pleased.",
  array([[0.75, 0.25]])),
 ("symbiotic kinship between organism and their nature. b'remains dead slaked to persist the same throughout  persist dead slaked to persist t/e same throughout '",
  [[0.5396552085876465, 0.4603447914123535]]),
 ("the cadaver of an being that has not been slaked b'remains dead slaked to remain the same throughout  cadaver dead slaked to remain t/e same throughout '",
  [[1.0, 0.0]])]